In [3]:
import torch, torchvision
from replace_Backbone import CSPdarkNetWithFPN

In [4]:
backbone = CSPdarkNetWithFPN()

In [5]:
sizes=((32,),(64,),(128,),(256,))
aspect_ratios=((0.5, 1.0, 2.0),)* len(sizes)
# anchor模板是怎样生成锚框的
for size, aspect_ratio in zip(sizes, aspect_ratios):
    print(size,aspect_ratio)
    size = torch.as_tensor(size)
    aspect_ratio = torch.as_tensor(aspect_ratio)
    print(size.shape, aspect_ratio.shape)
    print(size[None,:].shape, aspect_ratio[:,None].shape)

# 根据anchor模板参数生成对应的anchors模板
# 在特征图的每一个pixel还原到原图上的区域
# 以区域的左上角点为中心生成对应模板的锚框
from typing import List
def generate_anchors(
    scales: List[int],
    aspect_ratios: List[float],
    dtype: torch.dtype = torch.float32,
    device: torch.device = torch.device("cpu"),
):
    scales = torch.as_tensor(scales, dtype=dtype, device=device)
    aspect_ratios = torch.as_tensor(aspect_ratios, dtype=dtype, device=device)
    h_ratios = torch.sqrt(aspect_ratios)
    w_ratios = 1 / h_ratios

    ws = (w_ratios[:, None] * scales[None, :]).view(-1)
    hs = (h_ratios[:, None] * scales[None, :]).view(-1)

    base_anchors = torch.stack([-ws, -hs, ws, hs], dim=1) / 2
    return base_anchors.round()
cell_anchors = [
    generate_anchors(
        scales, aspect_ratio) for scales, aspect_ratio in zip(sizes, aspect_ratios)
    ]
cell_anchors

(32,) (0.5, 1.0, 2.0)
torch.Size([1]) torch.Size([3])
torch.Size([1, 1]) torch.Size([3, 1])
(64,) (0.5, 1.0, 2.0)
torch.Size([1]) torch.Size([3])
torch.Size([1, 1]) torch.Size([3, 1])
(128,) (0.5, 1.0, 2.0)
torch.Size([1]) torch.Size([3])
torch.Size([1, 1]) torch.Size([3, 1])
(256,) (0.5, 1.0, 2.0)
torch.Size([1]) torch.Size([3])
torch.Size([1, 1]) torch.Size([3, 1])


[tensor([[-23., -11.,  23.,  11.],
         [-16., -16.,  16.,  16.],
         [-11., -23.,  11.,  23.]]),
 tensor([[-45., -23.,  45.,  23.],
         [-32., -32.,  32.,  32.],
         [-23., -45.,  23.,  45.]]),
 tensor([[-91., -45.,  91.,  45.],
         [-64., -64.,  64.,  64.],
         [-45., -91.,  45.,  91.]]),
 tensor([[-181.,  -91.,  181.,   91.],
         [-128., -128.,  128.,  128.],
         [ -91., -181.,   91.,  181.]])]

In [6]:
# 原图尺寸
image_size = (320,640)
# 经过FPN提取的特征图
feature_maps = backbone(torch.rand(4,3,320,640))
grid_sizes = list([v.shape[-2:] for v in feature_maps.values()])
print('grid_sizes')
for i in grid_sizes:
    print(i
    )
print('***')
# 获取每一个特征图的每一个像素坐标相对于原图的坐标的比例
strides = [
    [
        torch.tensor(image_size[0] // g[0], dtype=torch.int64),
        torch.tensor(image_size[1] // g[1], dtype=torch.int64),
    ]
    for g in grid_sizes
]
print('strides')
for i in strides:
    print(i)

grid_sizes
torch.Size([40, 80])
torch.Size([20, 40])
torch.Size([10, 20])
torch.Size([5, 10])
***
strides
[tensor(8), tensor(8)]
[tensor(16), tensor(16)]
[tensor(32), tensor(32)]
[tensor(64), tensor(64)]


In [7]:
# 特征图数量和布局、anchor数量要一致
# 每一个尺度的特征图都对应不同的布局，使用对应的anchor模板，以下标对应
assert (len(grid_sizes) == len(strides) == len(cell_anchors))

In [8]:
anchors_over_all_feature_maps = []
#特征图大小，特征图相对于原图的步距，每一个网格的anchor模板
for size, stride, base_anchors in zip(grid_sizes, strides, cell_anchors):
    print("特征图的大小", "还原回原图的步距", "对应这个特征图的anchor模板","生成对应数量的锚框")
    print(size, stride, base_anchors, size[0]*size[1]*len(cell_anchors))

    grid_height, grid_width = size
    stride_height, stride_width = stride
    device = base_anchors.device

    # For output anchor, compute [x_center, y_center, x_center, y_center]
    # 计算特征图每一个pixel还原回原图时的中心坐标位置x,y
    shifts_x = torch.arange(0, grid_width, dtype=torch.int32, device=device) * stride_width
    shifts_y = torch.arange(0, grid_height, dtype=torch.int32, device=device) * stride_height
    
    shift_y, shift_x = torch.meshgrid(shifts_y, shifts_x, indexing="ij")
    shift_x = shift_x.reshape(-1)
    shift_y = shift_y.reshape(-1)
    shifts = torch.stack((shift_x, shift_y, shift_x, shift_y), dim=1)

    # For every (base anchor, output anchor) pair,
    # offset each zero-centered base anchor by the center of the output anchor.
    anchors_over_all_feature_maps.append((shifts.view(-1, 1, 4) + base_anchors.view(1, -1, 4)).reshape(-1, 4))

特征图的大小 还原回原图的步距 对应这个特征图的anchor模板 生成对应数量的锚框
torch.Size([40, 80]) [tensor(8), tensor(8)] tensor([[-23., -11.,  23.,  11.],
        [-16., -16.,  16.,  16.],
        [-11., -23.,  11.,  23.]]) 12800
特征图的大小 还原回原图的步距 对应这个特征图的anchor模板 生成对应数量的锚框
torch.Size([20, 40]) [tensor(16), tensor(16)] tensor([[-45., -23.,  45.,  23.],
        [-32., -32.,  32.,  32.],
        [-23., -45.,  23.,  45.]]) 3200
特征图的大小 还原回原图的步距 对应这个特征图的anchor模板 生成对应数量的锚框
torch.Size([10, 20]) [tensor(32), tensor(32)] tensor([[-91., -45.,  91.,  45.],
        [-64., -64.,  64.,  64.],
        [-45., -91.,  45.,  91.]]) 800
特征图的大小 还原回原图的步距 对应这个特征图的anchor模板 生成对应数量的锚框
torch.Size([5, 10]) [tensor(64), tensor(64)] tensor([[-181.,  -91.,  181.,   91.],
        [-128., -128.,  128.,  128.],
        [ -91., -181.,   91.,  181.]]) 200


In [9]:
for i in anchors_over_all_feature_maps:
    print(i.shape)

torch.Size([9600, 4])
torch.Size([2400, 4])
torch.Size([600, 4])
torch.Size([150, 4])
